In [2]:
#importing necessary libraries and modules
from Core_Game_Parts import *
import pygame
import time


In [3]:
def run_simulation(
    start_x=DEFAULT_START_X,
    start_y=DEFAULT_START_Y,
    start_angle=DEFAULT_START_ANGLE,
    start_speed=DEFAULT_START_SPEED
):
    """Runs the car simulation manually by the user.

    Args:
        start_x (int, optional): The starting x-coordinate of the car. Defaults to DEFAULT_START_X.
        start_y (int, optional): The starting y-coordinate of the car. Defaults to DEFAULT_START_Y.
        start_angle (float, optional): The starting angle of the car. Defaults to DEFAULT_START_ANGLE.
        start_speed (float, optional): The starting speed of the car. Defaults to DEFAULT_START_SPEED.
    """
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Car Simulation")

    track_image = pygame.image.load(TRACK_IMAGE_PATH).convert()
    track_surface = track_image.copy()

    car = Car(CAR_IMAGE_PATH, start_x, start_y, start_angle, start_speed)

    screen.fill(BG_COLOR)
    screen.blit(track_image, (0, 0))

    CAR_ROAD_COLOR = track_surface.get_at((start_x, start_y))[:3]

  

    lap_start_time=0
    best_lap_time=float('inf')
    current_lap_time=0
    lap_started=False
    car_on_finish=False

    car_max_speed = 0
    
    #checkpoints
    checkpoints=[]
    
    for x,y,w,h,a in checkpoint_data:
        base_surface=pygame.Surface((w,h),pygame.SRCALPHA)
        base_surface.fill((255,0,255,100))
        rotated_surface=pygame.transform.rotate(base_surface,a)
        rect=rotated_surface.get_rect(center=(x,y))
        mask=pygame.mask.from_surface(rotated_surface)
        checkpoints.append({'surface': rotated_surface, 'rect': rect, 'mask': mask})
    current_checkpoint=0

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        keys = pygame.key.get_pressed()
        if car.speed > 0:
            car.speed -= FRICTION
        if car.speed < 0:
            car.speed += FRICTION
        if abs(car.speed) < FRICTION:
            car.speed = 0
        if keys[pygame.K_DOWN]:
            car.speed -= BRAKE_FORCE
        if keys[pygame.K_UP]:
            if car.speed < MAX_SPEED:
                car.speed += ACCELERATION
                
        if car.speed > 0.1:
            speed_factor = car.speed / MAX_SPEED
            turn_range = MAX_TURN_ANGLE - MIN_TURN_ANGLE
            dynamic_turn_angle = MAX_TURN_ANGLE - (speed_factor * turn_range)
        
            if keys[pygame.K_LEFT]:
                car.angle += dynamic_turn_angle
            if keys[pygame.K_RIGHT]:
                car.angle -= dynamic_turn_angle

        screen.blit(track_image, (0, 0))

        font = pygame.font.Font(None, 22)
        text_surface = font.render(f"Speed: {car.speed:.2f}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 50))

        # Ray casting
        distance, sensor_endpoint = ray_casting(car, track_surface)
        text_surface = font.render(f"Distances: {distance}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 80))
        for endpoint in sensor_endpoint:
            pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), endpoint, 1)

        car.move()
        
        #cheking checkpoints
        if current_checkpoint < len(checkpoints):
            target = checkpoints[current_checkpoint]

            # compute offset of target mask relative to car mask
            offset_x = target['rect'].left - car.rect.left
            offset_y = target['rect'].top  - car.rect.top


            if (offset_y==0 and -60<offset_x<60) or (offset_x==0 and -60<offset_y<60):
                current_checkpoint += 1
                print(f"Checkpoint reached! {current_checkpoint}")

        
        car.draw(screen)
        
        for i, cp in enumerate(checkpoints):
          screen.blit(cp['surface'], cp['rect'])
          if i == current_checkpoint:
              pygame.draw.rect(screen, (0, 255, 0), cp['rect'], 3)
        
        #lap timing (collision detection)
        is_colliding=car.get_rect().colliderect(finish_line_rect)
        if is_colliding and not car_on_finish:
            car_on_finish=True
            if lap_started and current_checkpoint == len(checkpoints):
                lap_time=time.time()-lap_start_time
                best_lap_time=min(best_lap_time,lap_time)
                lap_start_time=time.time()
                current_checkpoint=0
            elif not lap_started:
                lap_started = True
                lap_start_time = time.time()
                current_checkpoint = 0
        if not is_colliding:
            car_on_finish=False
            
        pygame.draw.rect(screen, (250, 100, 50), finish_line_rect)
        

        # Collision detection (including out-of-bounds)
        crashed = False
        car_center_pos = (int(car.x), int(car.y))
        if not (0 <= car.x < SCREEN_WIDTH and 0 <= car.y < SCREEN_HEIGHT):
            crashed = True
        else:
            try:
                pixel_color = track_surface.get_at(car_center_pos)[:3]
                if pixel_color != CAR_ROAD_COLOR:
                    crashed = True
            except IndexError:
                crashed = True

        if crashed:
            lap_started=False
            current_lap_time=0 
            current_checkpoint=0
            text_surface = font.render("Car has crashed!", True, (255, 255, 255))
            screen.blit(text_surface, (50, 110))
            pygame.display.update()
            pygame.time.wait(1000)  # Show message for 1 second
            car.x, car.y = start_x, start_y
            car.angle = start_angle
            car.speed = start_speed

        if car.speed > car_max_speed:
            car_max_speed = car.speed

        if lap_started:
            current_lap_time = time.time() - lap_start_time
            current_lap_str = f"Lap: {current_lap_time:.2f}s"
        else:
            current_lap_str = "Lap: 0.00s"                                           
        best_lap_str = f"Best: {best_lap_time:.2f}s" if best_lap_time != float('inf') else "Best: N/A"
        text_surface = font.render(current_lap_str, True, (255, 255, 255))            
        screen.blit(text_surface, (50, 500))
        best_text = font.render(best_lap_str, True, (255, 255, 255))
        screen.blit(best_text, (50, 530))
        text_surface = font.render(f"Max speed: {car_max_speed:.2f}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 470))

        pygame.display.update()
        clock.tick(60)

    pygame.quit()

In [4]:
if __name__ == "__main__":
    run_simulation()

Checkpoint reached! 1
Checkpoint reached! 2
Checkpoint reached! 3
Checkpoint reached! 1
Checkpoint reached! 1
Checkpoint reached! 2
Checkpoint reached! 1
Checkpoint reached! 1
Checkpoint reached! 2
Checkpoint reached! 3
